In [2]:
import pandas as pd
import gzip as gz
import sqlite3
from zipfile import ZipFile

In [3]:
imdb = 'data/im.db.zip'
with ZipFile(imdb, 'r') as zip:
    zip.extractall('data/imdb_unzipped')

conn = sqlite3.connect('data/imdb_unzipped/im.db')
        
q = """
SELECT *
FROM sqlite_master
"""
pd.read_sql(q, conn)

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [4]:
q = """
SELECT *
FROM movie_basics
ORDER BY start_year
"""
pd.read_sql(q, conn)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0146592,Pál Adrienn,Pál Adrienn,2010,136.0,Drama
1,tt0154039,So Much for Justice!,Oda az igazság,2010,100.0,History
2,tt0162942,Children of the Green Dragon,A zöld sárkány gyermekei,2010,89.0,Drama
3,tt0230212,The Final Journey,The Final Journey,2010,120.0,Drama
4,tt0312305,Quantum Quest: A Cassini Space Odyssey,Quantum Quest: A Cassini Space Odyssey,2010,45.0,"Adventure,Animation,Sci-Fi"
...,...,...,...,...,...,...
146139,tt6149054,Fantastic Beasts and Where to Find Them 5,Fantastic Beasts and Where to Find Them 5,2024,NaN,"Adventure,Family,Fantasy"
146140,tt3095356,Avatar 4,Avatar 4,2025,NaN,"Action,Adventure,Fantasy"
146141,tt10300398,Untitled Star Wars Film,Untitled Star Wars Film,2026,NaN,Fantasy
146142,tt5637536,Avatar 5,Avatar 5,2027,NaN,"Action,Adventure,Fantasy"


In [5]:
#Include movies released over a 10 year period from 2012 to 2021
#Dropped redundant original_title column.

q = """
SELECT movie_id, primary_title AS 'Movie Title', start_year, runtime_minutes, genres
FROM movie_basics
WHERE start_year BETWEEN 2012 AND 2021
ORDER BY start_year 
"""
movie_clean = pd.read_sql(q, conn)

In [6]:
movie_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121352 entries, 0 to 121351
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         121352 non-null  object 
 1   Movie Title      121352 non-null  object 
 2   start_year       121352 non-null  int64  
 3   runtime_minutes  93709 non-null   float64
 4   genres           116821 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 4.6+ MB


In [7]:
movie_clean.head()

,movie_id,Movie Title,start_year,runtime_minutes,genres
0,tt0139613,O Silêncio,2012,NaN,"Documentary,History"
1,tt0144449,Nema aviona za Zagreb,2012,82.0,Biography
2,tt0249516,Foodfight!,2012,91.0,"Action,Animation,Comedy"
3,tt0250404,Godfather,2012,NaN,"Crime,Drama"
4,tt0262218,Akakis mogzauroba,2012,44.0,Documentary


In [12]:
movie_clean['genres'].value_counts()

Documentary                  27028
Drama                        17763
Comedy                        7640
Horror                        3743
Comedy,Drama                  2917
                             ...  
Documentary,Horror,Sci-Fi        1
Biography,Music,Romance          1
Horror,Musical,Thriller          1
Music,Mystery                    1
Action,Fantasy,Sport             1
Name: genres, Length: 1046, dtype: int64

In [9]:
#List of genres in movie_clean
list_of_genres = ['Action',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Sport',
 'Talk-Show',
 'Thriller',
 'War',
 'Western']

In [58]:
#Matching test with Movie Title in BudgetsDF
match_test = movie_clean[(movie_clean['Movie Title'] == 'Avengers: Age of Ultron')]
match_test

,movie_id,Movie Title,start_year,runtime_minutes,genres
44848,tt2395427,Avengers: Age of Ultron,2015,141.0,"Action,Adventure,Sci-Fi"


In [13]:
movie_clean

,movie_id,Movie Title,start_year,runtime_minutes,genres
0,tt0139613,O Silêncio,2012,NaN,"Documentary,History"
1,tt0144449,Nema aviona za Zagreb,2012,82.0,Biography
2,tt0249516,Foodfight!,2012,91.0,"Action,Animation,Comedy"
3,tt0250404,Godfather,2012,NaN,"Crime,Drama"
4,tt0262218,Akakis mogzauroba,2012,44.0,Documentary
...,...,...,...,...,...
121347,tt9412786,FlabberGasted,2021,NaN,Animation
121348,tt9598520,Golmaal 5,2021,NaN,"Adventure,Comedy,Mystery"
121349,tt9603212,Mission: Impossible 7,2021,NaN,"Action,Adventure,Thriller"
121350,tt9848626,Hotel Transylvania 4,2021,NaN,"Adventure,Animation,Comedy"


In [17]:
with gz.open('data/tn.movie_budgets.csv.gz') as f:
    budgets = pd.read_csv(f,encoding='latin1')

# Remove extraneous '$' and ',' symbols
budgets['production_budget'] = budgets['production_budget'].str.replace(',', '')
budgets['production_budget'] = budgets['production_budget'].str.replace("$", '')

budgets['domestic_gross'] = budgets['domestic_gross'].str.replace(",", '')
budgets['domestic_gross'] = budgets['domestic_gross'].str.replace("$", '')

budgets['worldwide_gross'] = budgets['worldwide_gross'].str.replace(",", '')
budgets['worldwide_gross'] = budgets['worldwide_gross'].str.replace("$", '')

# Convert money columns to integers
cash_columns = ['production_budget', 'domestic_gross', 'worldwide_gross']
budgets[cash_columns] = budgets[cash_columns].apply(pd.to_numeric)

# Convert to datetime
budgets['release_date'] = pd.to_datetime(budgets['release_date'])

# Create column to evaluate profitability
budgets['net_earnings'] = budgets['worldwide_gross'] - budgets['domestic_gross']

# Filter budgets to include only those in our target range
recent = budgets.loc[(budgets['release_date'] >= '2012-01-01')]
recent

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,net_earnings
2,3,2019-06-07,Dark Phoenix,350000000,42762350,149762350,107000000
3,4,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1403013963,944008095
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,696540365
5,6,2015-12-18,Star Wars Ep. VII: The Force Awakens,306000000,936662225,2053311220,1116648995
6,7,2018-04-27,Avengers: Infinity War,300000000,678815482,2048134200,1369318718
...,...,...,...,...,...,...,...
5761,62,2014-12-31,Stories of Our Lives,15000,0,0,0
5771,72,2015-05-19,Family Motocross,10000,0,0,0
5772,73,2012-01-13,Newlyweds,9000,4584,4584,0
5777,78,2018-12-31,Red 11,7000,0,0,0


In [29]:
#Merged Movies and BudgetsDF
Genre_Budget = movie_clean.merge(recent, left_on='Movie Title', right_on='movie', how='inner')

#Dropped redundant movie and start_year, and id columns
Genre_Budget = Genre_Budget.drop(['movie', 'start_year', 'id'], axis=1)
Genre_Budget.tail()

,movie_id,Movie Title,runtime_minutes,genres,release_date,production_budget,domestic_gross,worldwide_gross,net_earnings
1888,tt8155288,Happy Death Day 2U,100.0,"Drama,Horror,Mystery",2019-02-13,9000000,28051045,64179495,36128450
1889,tt8266310,Blinded by the Light,117.0,"Biography,Comedy,Drama",2019-08-14,15000000,0,0,0
1890,tt9024106,Unplanned,106.0,"Biography,Drama",2019-03-29,6000000,18107621,18107621,0
1891,tt10329540,Rogue City,NaN,None,2019-12-31,13000000,0,0,0
1892,tt7504726,Call of the Wild,NaN,"Adventure,Animation,Family",2020-02-21,82000000,0,0,0


In [30]:
Genre_Budget.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1893 entries, 0 to 1892
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   movie_id           1893 non-null   object        
 1   Movie Title        1893 non-null   object        
 2   runtime_minutes    1748 non-null   float64       
 3   genres             1873 non-null   object        
 4   release_date       1893 non-null   datetime64[ns]
 5   production_budget  1893 non-null   int64         
 6   domestic_gross     1893 non-null   int64         
 7   worldwide_gross    1893 non-null   int64         
 8   net_earnings       1893 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(4), object(3)
memory usage: 147.9+ KB
